In [3]:
import os
from os import listdir
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from sklearn.metrics import classification_report
import json
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from vanilla_gan import GAN
import warnings
warnings.filterwarnings('ignore')

results = {}

def append_record(record):
    with open('./results/individual_results_vanilla_gan.txt', 'w') as f:
        f.write(str(record))
        f.write(os.linesep)

path = "./data/vanillaGAN/"
for name in os.listdir(path) :
  if name.endswith(".csv") :
    for i in range(2) :
      print()
      print()
      print(name, i+1)
      print()
      print()
      file = pd.read_csv(path+name, header=None)
      file.loc[file[file.columns[-1]]>=1, file.columns[-1]] = 1
      scaler = MinMaxScaler()
      scaler.fit(file)
      file = scaler.transform(file)
      file = pd.DataFrame(file)
      if i == 0:
        results[name] = {}
      results[name][i+1] = {"vanilla_gan" : {}}

      y = file[file.columns[-1]]
      x = file.drop([file.columns[-1]], axis=1)
      
      train_x, test_x, train_y, test_y = tts(x, y, test_size=0.3, shuffle=True, stratify=y)
      train_y = np.resize(train_y, (train_y.shape[0], 1))
      train = np.concatenate([train_x, train_y], axis=1)
      train = pd.DataFrame(train)
      test_y = np.resize(test_y, (test_y.shape[0], 1))
      test = np.concatenate([test_x, test_y], axis=1)
      test = pd.DataFrame(test)
      train_path = path + "train/train_"+str(i+1)+"_"+name
      test_path = path + "test/test_"+str(i+1)+"_"+name
      train.to_csv(train_path, index=False, header=None)
      test.to_csv(test_path, index=False, header=None)

      train = pd.read_csv(train_path, header=None)
      test = pd.read_csv(test_path, header=None)

      test_y = test[test.columns[-1]]
      test_x = test.drop([test.columns[-1]], axis=1)

      nums = Counter(train[train.columns[-1]])
      if nums[0] > nums[1] :
        minor = 1
      else :
        minor = 0

      minority = train[train[train.columns[-1]] == minor]
      minority = minority.drop([minority.columns[-1]], axis=1)

      data_cols = minority.columns
      #Define the GAN and training parameters
      noise_dim = 32
      dim = 128
      batch_size = 32

      log_step = 100
      epochs = 5000+1
      learning_rate = 5e-4
      models_dir = 'model'
      gan_args = [batch_size, learning_rate, noise_dim, minority.shape[1], dim]
      train_args = ['', epochs, log_step]

      model = GAN
      synthesizer = model(gan_args)
      synthesizer.train(minority, train_args)

      z = np.random.normal(size=(abs(nums[0]-nums[1]), noise_dim))
      g_z = synthesizer.generator.predict(z)
      gen_samples = pd.DataFrame(g_z)
      gen_samples[len(train.columns)-1] = minor

      hybrid = pd.concat([gen_samples, train], axis=0, ignore_index=True)
      hybrid.to_csv(path + "hybrid_vanilla_gan_"+str(i+1)+"_"+name, index=False, header=None)

      vanilla_gan_y = hybrid[hybrid.columns[-1]]
      vanilla_gan_x = hybrid.drop([hybrid.columns[-1]], axis=1)

      # CLASSIFICATION
      clf = KNeighborsClassifier(5)
      clf.fit(vanilla_gan_x, vanilla_gan_y)
      pred_y = clf.predict(test_x)
      report = classification_report(test_y, pred_y, output_dict=True)
      disp = plot_confusion_matrix(clf, test_x, test_y,
                                    display_labels=[0, 1],
                                    cmap=plt.cm.Blues)
      precision = report['macro avg']['precision']
      recall = report['macro avg']['recall']
      score = report['accuracy']
      f1 = report['macro avg']['f1-score']
      weighted_precision = report['weighted avg']['precision']
      weighted_recall = report['weighted avg']['recall']
      weighted_f1 = report['weighted avg']['f1-score']
      auc = roc_auc_score(test_y, clf.predict_proba(test_x)[:, 1])

      tp = disp.confusion_matrix[1][1]
      fp = disp.confusion_matrix[0][1]
      fn = disp.confusion_matrix[1][0]
      tn = disp.confusion_matrix[0][0]
      results[name][i+1]["vanilla_gan"]['knn'] = [score, precision, recall, f1, auc, tp, fp, tn, fn, weighted_precision, weighted_recall, weighted_f1]

      clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=len(vanilla_gan_x.columns))
      clf.fit(vanilla_gan_x, vanilla_gan_y)
      pred_y = clf.predict(test_x)
      report = classification_report(test_y, pred_y, output_dict=True)
      disp = plot_confusion_matrix(clf, test_x, test_y,
                                    display_labels=[0, 1],
                                    cmap=plt.cm.Blues)
      precision = report['macro avg']['precision']
      recall = report['macro avg']['recall']
      score = report['accuracy']
      f1 = report['macro avg']['f1-score']
      weighted_precision = report['weighted avg']['precision']
      weighted_recall = report['weighted avg']['recall']
      weighted_f1 = report['weighted avg']['f1-score']
      auc = roc_auc_score(test_y, clf.predict_proba(test_x)[:, 1])

      tp = disp.confusion_matrix[1][1]
      fp = disp.confusion_matrix[0][1]
      fn = disp.confusion_matrix[1][0]
      tn = disp.confusion_matrix[0][0]
      results[name][i+1]["vanilla_gan"]['rf'] = [score, precision, recall, f1, auc, tp, fp, tn, fn, weighted_precision, weighted_recall, weighted_f1]


      clf = DecisionTreeClassifier(max_depth=5)
      clf.fit(vanilla_gan_x, vanilla_gan_y)
      pred_y = clf.predict(test_x)
      report = classification_report(test_y, pred_y, output_dict=True)
      disp = plot_confusion_matrix(clf, test_x, test_y,
                                    display_labels=[0, 1],
                                    cmap=plt.cm.Blues)
      precision = report['macro avg']['precision']
      recall = report['macro avg']['recall']
      score = report['accuracy']
      f1 = report['macro avg']['f1-score']
      weighted_precision = report['weighted avg']['precision']
      weighted_recall = report['weighted avg']['recall']
      weighted_f1 = report['weighted avg']['f1-score']
      auc = roc_auc_score(test_y, clf.predict_proba(test_x)[:, 1])

      tp = disp.confusion_matrix[1][1]
      fp = disp.confusion_matrix[0][1]
      fn = disp.confusion_matrix[1][0]
      tn = disp.confusion_matrix[0][0]
      results[name][i+1]["vanilla_gan"]['dt'] = [score, precision, recall, f1, auc, tp, fp, tn, fn, weighted_precision, weighted_recall, weighted_f1]


      clf = GaussianNB()
      clf.fit(vanilla_gan_x, vanilla_gan_y)
      pred_y = clf.predict(test_x)
      report = classification_report(test_y, pred_y, output_dict=True)
      disp = plot_confusion_matrix(clf, test_x, test_y,
                                    display_labels=[0, 1],
                                    cmap=plt.cm.Blues)
      precision = report['macro avg']['precision']
      recall = report['macro avg']['recall']
      score = report['accuracy']
      f1 = report['macro avg']['f1-score']
      weighted_precision = report['weighted avg']['precision']
      weighted_recall = report['weighted avg']['recall']
      weighted_f1 = report['weighted avg']['f1-score']
      auc = roc_auc_score(test_y, clf.predict_proba(test_x)[:, 1])

      tp = disp.confusion_matrix[1][1]
      fp = disp.confusion_matrix[0][1]
      fn = disp.confusion_matrix[1][0]
      tn = disp.confusion_matrix[0][0]
      results[name][i+1]["vanilla_gan"]['nb'] = [score, precision, recall, f1, auc, tp, fp, tn, fn, weighted_precision, weighted_recall, weighted_f1]


      clf = LogisticRegression()
      clf.fit(vanilla_gan_x, vanilla_gan_y)
      pred_y = clf.predict(test_x)
      report = classification_report(test_y, pred_y, output_dict=True)
      disp = plot_confusion_matrix(clf, test_x, test_y,
                                    display_labels=[0, 1],
                                    cmap=plt.cm.Blues)
      precision = report['macro avg']['precision']
      recall = report['macro avg']['recall']
      score = report['accuracy']
      f1 = report['macro avg']['f1-score']
      weighted_precision = report['weighted avg']['precision']
      weighted_recall = report['weighted avg']['recall']
      weighted_f1 = report['weighted avg']['f1-score']
      auc = roc_auc_score(test_y, clf.predict_proba(test_x)[:, 1])

      tp = disp.confusion_matrix[1][1]
      fp = disp.confusion_matrix[0][1]
      fn = disp.confusion_matrix[1][0]
      tn = disp.confusion_matrix[0][0]
      results[name][i+1]["vanilla_gan"]['lr'] = [score, precision, recall, f1, auc, tp, fp, tn, fn, weighted_precision, weighted_recall, weighted_f1]
      append_record(results)

dict1 = results
results = {}

nan = 0
for nam in dict1 :
    if(len(dict1[nam]) < 10) :
        print("there are not 10 iterations for " + nam + ", hence, results will not be calculated for it")
        continue
    results[nam] = {"vanilla_gan":{}}
    for itr in dict1[nam] :
          for typ in dict1[nam][itr] :
                for classifier in dict1[nam][itr][typ] :
                      if typ not in results[nam] :
                            results[nam][typ] = {}
                      if classifier not in results[nam][typ] :
                            results[nam][typ][classifier] = [[], [], [], [], [], [], [], [], [], [], [], []]

                      i = 0
                      for value in dict1[nam][itr][typ][classifier] :
                            results[nam][typ][classifier][i].append(value)
                            i += 1

final_results = {}
for name in results :
      final_results[name] = {"vanilla_gan":{}}
      for typ in results[name] :
            for classifier in results[name][typ] :
                  final_results[name][typ][classifier] = []
                  for values in results[name][typ][classifier] :
                        final_results[name][typ][classifier].append(sum(values)/len(values))

with open('./results/averaged_results_vanilla_gan.txt', 'w') as f:
    f.write(str(final_results))

print("Results are printed in below order.")
print("accuracy, precision, recall, f1-score, auc, tp, fp, tn, fn, weighted_precision, weighted_recall, weighted_f1")
dict1 = final_results
for name in sorted(dict1) :
      print(name)
      for typ in dict1[name] :
            for classifier in dict1[name][typ] :
                  print(classifier)
                  for value in dict1[name][typ][classifier] :
                        print(round(value, 4), end = " ")
                  print()
            print()
      print()



CM1.csv 1


1/1 [==============================] - 0s 265ms/step
0 [D loss: 0.711498, acc.: 9.38%] [G loss: 0.686070]
1/1 [==============================] - 0s 17ms/step
1 [D loss: 0.694817, acc.: 51.56%] [G loss: 0.691908]
1/1 [==============================] - 0s 16ms/step
2 [D loss: 0.687136, acc.: 51.56%] [G loss: 0.697350]
1/1 [==============================] - 0s 14ms/step
3 [D loss: 0.695383, acc.: 50.00%] [G loss: 0.705501]
1/1 [==============================] - 0s 18ms/step
4 [D loss: 0.704674, acc.: 51.56%] [G loss: 0.719258]
1/1 [==============================] - 0s 14ms/step
5 [D loss: 0.719580, acc.: 51.56%] [G loss: 0.727804]
1/1 [==============================] - 0s 17ms/step
6 [D loss: 0.711085, acc.: 45.31%] [G loss: 0.808405]
1/1 [==============================] - 0s 14ms/step
7 [D loss: 0.673353, acc.: 53.12%] [G loss: 0.923640]
1/1 [==============================] - 0s 15ms/step
8 [D loss: 0.645023, acc.: 50.00%] [G loss: 1.020893]
1/1 [===========================

1/1 [==============================] - 0s 18ms/step
77 [D loss: 0.660158, acc.: 62.50%] [G loss: 0.726573]
1/1 [==============================] - 0s 17ms/step
78 [D loss: 0.666131, acc.: 50.00%] [G loss: 0.718733]
1/1 [==============================] - 0s 15ms/step
79 [D loss: 0.669373, acc.: 34.38%] [G loss: 0.707838]
1/1 [==============================] - 0s 15ms/step
80 [D loss: 0.673179, acc.: 34.38%] [G loss: 0.681670]
1/1 [==============================] - 0s 18ms/step
81 [D loss: 0.681760, acc.: 37.50%] [G loss: 0.673253]
1/1 [==============================] - 0s 16ms/step
82 [D loss: 0.681214, acc.: 37.50%] [G loss: 0.678302]
1/1 [==============================] - 0s 16ms/step
83 [D loss: 0.693634, acc.: 39.06%] [G loss: 0.684255]
1/1 [==============================] - 0s 20ms/step
84 [D loss: 0.685329, acc.: 40.62%] [G loss: 0.690622]
1/1 [==============================] - 0s 16ms/step
85 [D loss: 0.676024, acc.: 51.56%] [G loss: 0.736785]
1/1 [==============================] 

1/1 [==============================] - 0s 16ms/step
153 [D loss: 0.697423, acc.: 34.38%] [G loss: 0.726048]
1/1 [==============================] - 0s 18ms/step
154 [D loss: 0.758226, acc.: 21.88%] [G loss: 0.643366]
1/1 [==============================] - 0s 16ms/step
155 [D loss: 0.795119, acc.: 15.62%] [G loss: 0.605415]
1/1 [==============================] - 0s 14ms/step
156 [D loss: 0.889877, acc.: 15.62%] [G loss: 0.710334]
1/1 [==============================] - 0s 15ms/step
157 [D loss: 0.777694, acc.: 15.62%] [G loss: 1.050295]
1/1 [==============================] - 0s 17ms/step
158 [D loss: 0.662968, acc.: 51.56%] [G loss: 1.510540]
1/1 [==============================] - 0s 17ms/step
159 [D loss: 0.646018, acc.: 51.56%] [G loss: 1.345858]
1/1 [==============================] - 0s 15ms/step
160 [D loss: 0.673958, acc.: 51.56%] [G loss: 1.241811]
1/1 [==============================] - 0s 16ms/step
161 [D loss: 0.693696, acc.: 51.56%] [G loss: 1.041627]
1/1 [=======================

1/1 [==============================] - 0s 14ms/step
229 [D loss: 0.693715, acc.: 43.75%] [G loss: 0.739827]
1/1 [==============================] - 0s 15ms/step
230 [D loss: 0.690788, acc.: 39.06%] [G loss: 0.751997]
1/1 [==============================] - 0s 15ms/step
231 [D loss: 0.739698, acc.: 26.56%] [G loss: 0.819833]
1/1 [==============================] - 0s 16ms/step
232 [D loss: 0.668321, acc.: 53.12%] [G loss: 0.922838]
1/1 [==============================] - 0s 14ms/step
233 [D loss: 0.690023, acc.: 48.44%] [G loss: 1.081754]
1/1 [==============================] - 0s 16ms/step
234 [D loss: 0.605765, acc.: 56.25%] [G loss: 1.191894]
1/1 [==============================] - 0s 16ms/step
235 [D loss: 0.642802, acc.: 54.69%] [G loss: 1.057498]
1/1 [==============================] - 0s 16ms/step
236 [D loss: 0.662704, acc.: 53.12%] [G loss: 0.828326]
1/1 [==============================] - 0s 15ms/step
237 [D loss: 0.705108, acc.: 50.00%] [G loss: 0.783626]
1/1 [=======================

1/1 [==============================] - 0s 20ms/step
305 [D loss: 0.676915, acc.: 59.38%] [G loss: 0.756947]
1/1 [==============================] - 0s 16ms/step
306 [D loss: 0.675240, acc.: 60.94%] [G loss: 0.743994]
1/1 [==============================] - 0s 15ms/step
307 [D loss: 0.670956, acc.: 60.94%] [G loss: 0.734542]
1/1 [==============================] - 0s 17ms/step
308 [D loss: 0.680699, acc.: 56.25%] [G loss: 0.749886]
1/1 [==============================] - 0s 16ms/step
309 [D loss: 0.682653, acc.: 53.12%] [G loss: 0.745670]
1/1 [==============================] - 0s 17ms/step
310 [D loss: 0.677565, acc.: 56.25%] [G loss: 0.744600]
1/1 [==============================] - 0s 14ms/step
311 [D loss: 0.678932, acc.: 54.69%] [G loss: 0.746855]
1/1 [==============================] - 0s 15ms/step
312 [D loss: 0.676495, acc.: 53.12%] [G loss: 0.748113]
1/1 [==============================] - 0s 15ms/step
313 [D loss: 0.680797, acc.: 53.12%] [G loss: 0.742761]
1/1 [=======================

...

1/1 [==============================] - 0s 16ms/step
4739 [D loss: 0.372098, acc.: 84.38%] [G loss: 1.886356]
1/1 [==============================] - 0s 16ms/step
4740 [D loss: 0.328563, acc.: 85.94%] [G loss: 2.235782]
1/1 [==============================] - 0s 20ms/step
4741 [D loss: 0.430426, acc.: 79.69%] [G loss: 2.528482]
1/1 [==============================] - 0s 21ms/step
4742 [D loss: 0.428360, acc.: 79.69%] [G loss: 2.114676]
1/1 [==============================] - 0s 18ms/step
4743 [D loss: 0.406497, acc.: 82.81%] [G loss: 1.808971]
1/1 [==============================] - 0s 19ms/step
4744 [D loss: 0.366339, acc.: 81.25%] [G loss: 2.426806]
1/1 [==============================] - 0s 16ms/step
4745 [D loss: 0.419117, acc.: 75.00%] [G loss: 1.993760]
1/1 [==============================] - 0s 19ms/step
4746 [D loss: 0.345131, acc.: 84.38%] [G loss: 2.420971]
1/1 [==============================] - 0s 19ms/step
4747 [D loss: 0.336532, acc.: 84.38%] [G loss: 1.767824]
1/1 [==============

1/1 [==============================] - 0s 16ms/step
4814 [D loss: 0.429709, acc.: 81.25%] [G loss: 2.169686]
1/1 [==============================] - 0s 16ms/step
4815 [D loss: 0.439607, acc.: 75.00%] [G loss: 2.054371]
1/1 [==============================] - 0s 17ms/step
4816 [D loss: 0.357678, acc.: 78.12%] [G loss: 2.020334]
1/1 [==============================] - 0s 15ms/step
4817 [D loss: 0.370781, acc.: 82.81%] [G loss: 1.728784]
1/1 [==============================] - 0s 17ms/step
4818 [D loss: 0.401069, acc.: 76.56%] [G loss: 2.756843]
1/1 [==============================] - 0s 13ms/step
4819 [D loss: 0.408359, acc.: 71.88%] [G loss: 2.202399]
1/1 [==============================] - 0s 16ms/step
4820 [D loss: 0.372129, acc.: 78.12%] [G loss: 2.256960]
1/1 [==============================] - 0s 15ms/step
4821 [D loss: 0.336630, acc.: 79.69%] [G loss: 1.716288]
1/1 [==============================] - 0s 15ms/step
4822 [D loss: 0.371854, acc.: 85.94%] [G loss: 1.842936]
1/1 [==============

1/1 [==============================] - 0s 20ms/step
4889 [D loss: 0.384180, acc.: 82.81%] [G loss: 1.739985]
1/1 [==============================] - 0s 17ms/step
4890 [D loss: 0.356673, acc.: 81.25%] [G loss: 2.070196]
1/1 [==============================] - 0s 16ms/step
4891 [D loss: 0.419054, acc.: 78.12%] [G loss: 1.667940]
1/1 [==============================] - 0s 18ms/step
4892 [D loss: 0.420524, acc.: 79.69%] [G loss: 1.871986]
1/1 [==============================] - 0s 14ms/step
4893 [D loss: 0.355193, acc.: 78.12%] [G loss: 2.131482]
1/1 [==============================] - 0s 15ms/step
4894 [D loss: 0.281173, acc.: 89.06%] [G loss: 1.720918]
1/1 [==============================] - 0s 17ms/step
4895 [D loss: 0.358624, acc.: 78.12%] [G loss: 1.718898]
1/1 [==============================] - 0s 16ms/step
4896 [D loss: 0.313102, acc.: 85.94%] [G loss: 2.066895]
1/1 [==============================] - 0s 14ms/step
4897 [D loss: 0.398818, acc.: 78.12%] [G loss: 2.175010]
1/1 [==============

1/1 [==============================] - 0s 18ms/step
4964 [D loss: 0.404843, acc.: 82.81%] [G loss: 2.047972]
1/1 [==============================] - 0s 16ms/step
4965 [D loss: 0.387708, acc.: 75.00%] [G loss: 1.541990]
1/1 [==============================] - 0s 17ms/step
4966 [D loss: 0.353972, acc.: 84.38%] [G loss: 2.166141]
1/1 [==============================] - 0s 16ms/step
4967 [D loss: 0.331041, acc.: 84.38%] [G loss: 1.992412]
1/1 [==============================] - 0s 14ms/step
4968 [D loss: 0.318914, acc.: 82.81%] [G loss: 2.038776]
1/1 [==============================] - 0s 14ms/step
4969 [D loss: 0.344657, acc.: 79.69%] [G loss: 1.854750]
1/1 [==============================] - 0s 16ms/step
4970 [D loss: 0.362889, acc.: 81.25%] [G loss: 2.077091]
1/1 [==============================] - 0s 15ms/step
4971 [D loss: 0.371242, acc.: 81.25%] [G loss: 2.182810]
1/1 [==============================] - 0s 15ms/step
4972 [D loss: 0.409500, acc.: 82.81%] [G loss: 2.072463]
1/1 [==============

NameError: name 'plot_confusion_matrix' is not defined

In [3]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.
